In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import os
import pytz
from google.cloud import bigquery
from datetime import datetime
import datetime as dt

In [2]:
project_name = os.getenv('GCP_PROJECT')
dataset_name = os.getenv('BQ_DS')
table_name= os.getenv('BQ_TABLE')
location_name= os.getenv('BQ_LOCATION')

client = bigquery.Client()

In [7]:
# Get today's date in the right timezone, usually UTC for BigQuery
end = (datetime.now(pytz.utc).date() - dt.timedelta(days=2))
dates = list(reversed([(end - dt.timedelta(days=i)).strftime('%Y-%m-%d') for i in range(7)]))
values = [2, 2, 5, 6, 3, 3, 3]
print(dates)
print(dict())

['2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13']
[2, 2, 5, 6, 3, 3, 3]


In [10]:
from google.cloud import bigquery
from datetime import datetime
import os
import pytz

# Initialize BigQuery client
client = bigquery.Client()

def bulk_update_or_insert_values(dates, values, project_id=None, dataset_id=None, table_id=None):
    if project_id is None:
        project_id = os.getenv('GCP_PROJECT')
    if dataset_id is None:
        dataset_id = os.getenv('BQ_DS')
    if table_id is None:
        table_id = os.getenv('BQ_TABLE')

    # List to store the operations status
    operation_results = []

    # Loop through each date and value
    for date, value in zip(dates, values):
        # Query to check existing data for the given date
        check_query = f"""
        SELECT value
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE DATE(date) = '{date}'
        """
        check_job = client.query(check_query)
        results = list(check_job)

        # Determine whether to update or insert based on presence of data
        if results:
            # Update the existing row
            operation_query = f"""
            UPDATE `{project_id}.{dataset_id}.{table_id}`
            SET value = {value}
            WHERE DATE(date) = '{date}'
            """
            operation_type = "Update"
        else:
            # Insert a new row
            operation_query = f"""
            INSERT `{project_id}.{dataset_id}.{table_id}` (date, value)
            VALUES ('{date}', {value})
            """
            operation_type = "Insert"

        # Execute the update or insert query
        operation_job = client.query(operation_query)
        operation_job.result()  # Wait for the job to complete

        # Record the operation result
        operation_results.append(f"{operation_type} successful for date {date}")

    return operation_results

# Example usage
# dates = ['2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13']
# values = [2, 2, 5, 6, 3, 3, 3]
dates = ['2024-05-6']
values = [2]

print(bulk_update_or_insert_values(dates, values))


['Insert successful for date 2024-05-6']
